In [1]:
from astropy.io import fits
from astropy.time import Time
from astroquery.jplhorizons import Horizons
from _mypath import thisdir
import numpy as np
import os

In [2]:
def get_path(rela_path):
    """link the relative path to the path of this dir
    to get an absolute path
    
    Inputs: relative path, string
    Outputs: absolute path, string
    """
    return os.path.join(thisdir, rela_path)

In [3]:
def make_obs_log(horizon_id,
                 input_path, 
                 map_type, 
                 output_path):
    """creat an observing log file before all work,
    to provide non-data info for every extension.
    
    Inputs: 1. Horizon ID of the object, string;
            2. path of the folder containing dirs of every observation, string;
            3. 'sk' or 'rw' or 'ex', expected to be 'sk', string;
            4. ouput path of the newly generated obs log, string
            
    Outputs: No, only generate an obs log file
    """
    #-----initiate a log file-----
    f = open(output_path, 'w')
    f.write('OBS_ID '       +
            'EXTENSION '    +
            'START.[UT] '   +
            'END.[UT] '     +
            'MIDTIME '      +
            'EXP_TIME.[s] ' +
            'FILTER '       +
            'HELIO.[AU] '   +
            'HELIOV.[km/s] '+
            'PHASE[deg]'    +
            '\n')
    #-----start a loop iterating over observations-----
    obs_list = os.listdir(input_path)
    obs_list.sort()
    for obs_id in obs_list:
        #-----find the only *sk* file for every obs-----
        map_dir_path = os.path.join(input_path, 
                                    obs_id+'/uvot/image/')
        map_file_list = os.listdir(map_dir_path)
        map_file = [x for x in map_file_list 
                   if (map_type in x)]
        if len(map_file) == 1:
            map_file = map_file[0]
        else:
            break
        map_file_path = os.path.join(map_dir_path, 
                                     map_file)
        #-----get the num of extensions-----
        #-----start a loop interating over extensions----
        hdul = fits.open(map_file_path)
        ext_num = len(hdul) - 1
        for ext_id in range(1, 1 + ext_num):
            #-----read observing info from header of every extension-----
            #-----and put the info into the log file-----
            ext_header = hdul[ext_id].header
            start_time = Time(ext_header['DATE-OBS'])
            end_time   = Time(ext_header['DATE-END'])
            exp_time   = ext_header['TELAPSE']
            dt         = end_time - start_time
            mid_time   = start_time + 1/2*dt
            filt       = ext_header['FILTER']
            f.write(  obs_id                + ' '
                    + f'{int(ext_id)}'      + ' '
                    + f'{start_time.value}' + ' '
                    + f'{end_time.value}'   + ' '
                    + f'{mid_time.value}'   + ' '
                    + f'{exp_time}'         + ' '
                    + filt                  + ' ')
            #-----read ephmerides info from Horizon API-----
            #-----and put the info into the log file-----
            obj = Horizons(id=horizon_id,  
                           epochs=mid_time.jd)
            eph = obj.ephemerides()[0]
            helio = eph['r']
            heliov = eph['r_rate']
            phase = eph['alpha']
            f.write(  f'{helio}'            + ' '
                    + f'{heliov}'           + ' '
                    + f'{phase}'            + '\n')
    f.close()
            

In [4]:
#-----Borisov observed by Swift UVOT in Sep 2019-----
horizon_id  = 90004423
input_rela  = '../data/Borisov_raw/'
input_path  = get_path(input_rela)
map_type    = 'sk'
output_rela = '../docs/obs-log_Borisov_ZX.txt'
output_path = get_path(output_rela)

make_obs_log(horizon_id,
             input_path,
             map_type,
             output_path)